In [94]:
#Automaton Class
class Automaton:
    def __init__(self, start_state, accept_state):
        self.start_state = start_state
        self.accept_state = accept_state
        self.transitions = []
    
    def add_transition(self, current_state, symbol, next_state):
        self.transitions.append(Transition(current_state, symbol, next_state))
    
    def display(self):
        print(f"Start: q{self.start_state}")
        print(f"Accept: q{self.accept_state}")
        for transition in sorted(self.transitions):
            print(transition)


In [93]:
#transition class for Automaton
class Transition:
    def __init__(self, current_state, symbol, next_state):
        self.current_state = current_state
        self.symbol = symbol
        self.next_state = next_state
    
    def __lt__(self, other):
        if self.current_state != other.current_state:
            return self.current_state < other.current_state
        elif self.symbol != other.symbol:
            return self.symbol < other.symbol
        else:
            return self.next_state < other.next_state
    
    def __str__(self):
        return f"(q{self.current_state}, {self.symbol}) -> q{self.next_state}"


In [104]:
#test of Automaton
my_automaton = Automaton(start_state=6, accept_state=7)
my_automaton.add_transition(1, 'a', 2)
my_automaton.add_transition(2, 'E', 3)
my_automaton.add_transition(3, 'E', 1)
my_automaton.add_transition(3, 'E', 7)
my_automaton.add_transition(4, 'b', 5)
my_automaton.add_transition(5, 'E', 7)
my_automaton.add_transition(6, 'E', 3)
my_automaton.add_transition(6, 'E', 4)
my_automaton.display()

Start: q6
Accept: q7
(q1, a) -> q2
(q2, E) -> q3
(q3, E) -> q1
(q3, E) -> q7
(q4, b) -> q5
(q5, E) -> q7
(q6, E) -> q3
(q6, E) -> q4


In [96]:
#Testing | operation
stack = []

num_of_states = 0
char = "a"
my_automaton = Automaton(start_state=num_of_states+1, accept_state=num_of_states+2)
my_automaton.add_transition( num_of_states+1, char, num_of_states+2)
my_automaton.display()
stack.append(my_automaton)
num_of_states += 2
print()
char = "b"
my_automaton = Automaton(start_state=num_of_states+1, accept_state=num_of_states+2)
my_automaton.add_transition( num_of_states+1, char, num_of_states+2)
my_automaton.display()
stack.append(my_automaton)
num_of_states += 2

automaton_a = stack.pop()
automaton_b = stack.pop()

automaton_a_or_b = Automaton(start_state=num_of_states+1, accept_state=num_of_states+2)
#all transitions of a
#all transitions of b
# epsilon tranistion from new start state to start state of each old nfa
# epsilon transition from each old acceot state to the new one

automaton_a_or_b.transitions.extend(automaton_a.transitions)
automaton_a_or_b.transitions.extend(automaton_b.transitions)

# Add epsilon transitions
automaton_a_or_b.add_transition(automaton_a_or_b.start_state, 'E', automaton_a.start_state)
automaton_a_or_b.add_transition(automaton_a_or_b.start_state, 'E', automaton_b.start_state)
automaton_a_or_b.add_transition(automaton_a.accept_state, 'E', automaton_a_or_b.accept_state)
automaton_a_or_b.add_transition(automaton_b.accept_state, 'E', automaton_a_or_b.accept_state)

# Display the final automaton

print()
automaton_a_or_b.display()


Start: q1
Accept: q2
(q1, a) -> q2

Start: q3
Accept: q4
(q3, b) -> q4

Start: q5
Accept: q6
(q1, a) -> q2
(q2, E) -> q6
(q3, b) -> q4
(q4, E) -> q6
(q5, E) -> q1
(q5, E) -> q3


In [103]:
#Working Version
#input regular expression in postfix
inputRE = "a*b|"

stack = []
operands = "abcde"
operators = "|&*"
epsilon = "E"
num_of_states = 0


for char in inputRE:
    if char == epsilon: continue
    if char in operands:
        my_automaton = Automaton(start_state=num_of_states+1, accept_state=num_of_states+2)
        my_automaton.add_transition(num_of_states+1, char, num_of_states+2)
        stack.append(my_automaton)
        num_of_states += 2
    elif char == "|":
        if len(stack) < 2: 
            print("bad expression")
            break
        automaton_a = stack.pop()
        automaton_b = stack.pop()

        automaton_a_or_b = Automaton(start_state=num_of_states+1, accept_state=num_of_states+2)
        #all transition of a and b
        automaton_a_or_b.transitions.extend(automaton_a.transitions)
        automaton_a_or_b.transitions.extend(automaton_b.transitions)
        
        # Add epsilon transitions
        automaton_a_or_b.add_transition(automaton_a_or_b.start_state, 'E', automaton_a.start_state)
        automaton_a_or_b.add_transition(automaton_a_or_b.start_state, 'E', automaton_b.start_state)
        automaton_a_or_b.add_transition(automaton_a.accept_state, 'E', automaton_a_or_b.accept_state)
        automaton_a_or_b.add_transition(automaton_b.accept_state, 'E', automaton_a_or_b.accept_state)

        stack.append(automaton_a_or_b)
        num_of_states += 2

    elif char == "*":
        if len(stack) < 1:
            print("bad expression")
            break

        automaton_a = stack.pop()
        automaton_star = Automaton(start_state=num_of_states+1, accept_state=num_of_states+1)
        automaton_star.transitions.extend(automaton_a.transitions)
        automaton_star.add_transition(automaton_star.start_state, "E", automaton_a.start_state)
        automaton_star.add_transition(automaton_a.accept_state, "E", automaton_star.start_state)
        stack.append(automaton_star)
        num_of_states += 1

    elif char == "&":
        if len(stack) < 2: 
            print("bad expression")
            break
        automaton_a = stack.pop()
        automaton_b = stack.pop()

        automaton_ab = Automaton(start_state=automaton_b.start_state, accept_state=automaton_a.accept_state)
        #all transition of a and b
        automaton_ab.transitions.extend(automaton_a.transitions)
        automaton_ab.transitions.extend(automaton_b.transitions)
        automaton_ab.add_transition(automaton_b.accept_state, "E", automaton_a.start_state)
        stack.append(automaton_ab)
        


##show final automaton
print("RE: "+inputRE)
final_automaton = stack.pop()
final_automaton.display()

RE: a*b|
Start: q6
Accept: q7
(q1, a) -> q2
(q2, E) -> q3
(q3, E) -> q1
(q3, E) -> q7
(q4, b) -> q5
(q5, E) -> q7
(q6, E) -> q3
(q6, E) -> q4
